# ADS Lead SQL Notebook
#### Created by [@HiramFleitas](http://twitter.com/hiramfleitas)
![Hiram](https://media.licdn.com/dms/image/C4E03AQFo7vvpLPrFUw/profile-displayphoto-shrink_200_200/0?e=1561593600&v=beta&t=ZGV1zj9kr_Q1eP25jW8SA7lj8I9nuiWZ-kgv9l5m5V4 "Hiram")

## Create database on **localhost** SQL Server 2019 ctp 2.4+

In [1]:
if db_id('FleitasArts') is null create database FleitasArts;
go

Commands completed successfully.

Total execution time: 00:00:00.4668331

## Switch DB Context and cleanout to start fresh


In [2]:
use FleitasArts
go
drop table if exists Ticket
drop table if exists Lead
drop table if exists CustomerAccount
drop table if exists Customer
go
set nocount on;

Commands completed successfully.

Total execution time: 00:00:00.0061084

Commands completed successfully.

Total execution time: 00:00:00.0021093

Commands completed successfully.

Total execution time: 00:00:00.0003328